In [2]:
# to clean and pre-process the data
# import data using pandas
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [39]:
# label the cluster map
# labels:
# region_hash, region_id

columns = ['region_hash', 'region_id']
# read the cluster map
cluster_map = pd.read_csv('../dataset/training_data/cluster_map/cluster_map', sep='\t', on_bad_lines='skip', header=None, names=columns)
print('cluster_map finished')

print('cluster_map.head(): \n', cluster_map.head())


# cluster_map.to_csv('../dataset/labeledData/cluster_map.csv', index=False)



cluster_map finished
cluster_map.head(): 
                         region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [87]:
# label the data
# label the orders data
# labels:
# order_id, driver_id, passenger_id, start_district_hash, dest_district_hash, price, time
columns = ['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time']

# read the orders data
orders_data = []
for f in glob.glob('../dataset/training_data/order_data/order_data_*'):
    # file name
    print('filename: ', f)
    df = pd.read_csv(f, sep='\t', on_bad_lines='skip', header=None, names=columns)
    orders_data.append(df)

print('orders_data finished')
orders_data = pd.concat(orders_data,  ignore_index=True)

# print('orders_data.head(): ', orders_data.head())
orders_data.to_csv('../dataset/labeledData/orders_data.csv', index=False)


filename:  ../dataset/training_data/order_data\order_data_2016-01-01
filename:  ../dataset/training_data/order_data\order_data_2016-01-02
filename:  ../dataset/training_data/order_data\order_data_2016-01-03


KeyboardInterrupt: 

In [41]:
# label the weather data
# labels:
# time, weather, temperature, pm25
columns = ['time', 'weather', 'temperature', 'pm25']

# print('weather_data.head(): \n', weather_data.head())


# # read the weather data
weather_data = []
for f in glob.glob('../dataset/training_data/weather_data/weather_data_*'):
    # file name
    print('filename: ', f)
    df = pd.read_csv(f, sep='\t', on_bad_lines='skip', header=None, names=columns)
    weather_data.append(df)

print('weather_data finished')
weather_data = pd.concat(weather_data, ignore_index=True)


# weather_data.to_csv('../dataset/labeledData/weather_data.csv', index=False)


filename:  ../dataset/training_data/weather_data\weather_data_2016-01-01
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-02
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-03
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-04
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-05
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-06
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-07
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-08
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-09
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-10
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-11
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-12
filename:  ../dataset/training_data/weather_data\weather_data_2016-01-13
filename:  ../dataset/training_data/weather_data\we

In [74]:
# label the poi data
# labels:
# region_hash, poi_id 
# 1st column: district_hash
# whole next column is: poi_id
columns = ['region_hash', 'poi_id']



# read the poi data
poi_data = pd.read_csv('../dataset/training_data/poi_data/poi_data', sep='\t', header=None, on_bad_lines='skip')

# extract the district_hash column and the POI ID columns
district_hash = poi_data.iloc[:, 0]
poi_ids = poi_data.iloc[:, 1:]

# combine all the POI IDs for each row into a list
poi_ids_list = poi_ids.apply(lambda x: x.tolist(), axis=1)

# combine the district_hash and poi_ids_list into a new DataFrame
labeled_poi_data = pd.concat([district_hash, poi_ids_list], axis=1)
labeled_poi_data.columns = ['region_hash', 'poi_ids']

# print the result
print(labeled_poi_data.head())

# convert the column of lists to a list of lists
# list_of_lists = df['list_col'].tolist()

labeled_poi_data.to_csv('../dataset/labeledData/poi_data.csv', index=False)



                        region_hash  \
0  74c1c25f4b283fa74a5514307b0d0278   
1  08f5b445ec6b29deba62e6fd8b0325a6   
2  4b7f6f4e2bf237b6cc58f57142bea5c0   
3  a814069db8d32f0fa6e188f41059c6e1   
4  8316146a6f78cc6d9f113f0390859417   

                                             poi_ids  
0  [1#11:2241, 1#10:249, 24:1245, 25:3652, 20:334...  
1  [20#7:249, 20#5:83, 2#7:166, 20#2:747, 20#1:99...  
2  [4#16:249, 24:913, 25:332, 20:4316, 22:415, 4:...  
3  [1#11:498, 24:332, 25:581, 20:5810, 22:2407, 4...  
4  [20#7:581, 20#5:83, 20#4:415, 20#2:166, 20#1:6...  


In [75]:
# read from the labeled data
cluster_map = pd.read_csv('../dataset/labeledData/cluster_map.csv')
orders_data = pd.read_csv('../dataset/labeledData/orders_data.csv')
weather_data = pd.read_csv('../dataset/labeledData/weather_data.csv')
poi_data = pd.read_csv('../dataset/labeledData/poi_data.csv')


In [76]:
# map time to time slot
# devide day in 10 min time slots (144 time slots)


# convert time to datetime
orders_data['time'] = pd.to_datetime(orders_data['time'])
weather_data['time'] = pd.to_datetime(weather_data['time'])

# map time to time slot
orders_data['time_slot'] = orders_data['time'].dt.hour * 6 + orders_data['time'].dt.minute // 10
weather_data['time_slot'] = weather_data['time'].dt.hour * 6 + weather_data['time'].dt.minute // 10
# remove the time column
orders_data = orders_data.drop(['time'], axis=1)
weather_data = weather_data.drop(['time'], axis=1)

# print(orders_data.head())
# print(weather_data.head())

In [77]:
# group the orders data by time slot 
# aggregate count the number of orders where driver_id = NULL
# this is supply demand deficit

orders_data_grouped = orders_data[orders_data['driver_id'].isnull()].groupby(['time_slot', '']).agg({'order_id': 'count'}).rename(columns={'order_id': 'order_gap'}).reset_index()

print(orders_data_grouped)


     time_slot  order_gap
0            0       5092
1            1       5678
2            2       6995
3            3       8847
4            4       9093
..         ...        ...
139        139       3463
140        140       3200
141        141       3620
142        142       4084
143        143       4039

[144 rows x 2 columns]


In [78]:
# group the weather data by time slot
# aggregate the mean of temperature and pm25
weather_data_grouped = weather_data.groupby(['time_slot']).agg({'temperature': 'mean', 'pm25': 'mean'}).reset_index()

print(weather_data_grouped)

     time_slot  temperature        pm25
0            0     6.548387  133.838710
1            1     6.176471  132.735294
2            2     6.117647  131.911765
3            3     6.088235  132.500000
4            4     5.972222  132.305556
..         ...          ...         ...
139        139     6.387097  132.967742
140        140     6.108108  129.540541
141        141     5.970588  127.941176
142        142     6.085714  125.971429
143        143     6.125000  125.812500

[144 rows x 3 columns]


In [79]:
# merge the orders data and weather data
orders_weather_data = pd.merge(orders_data_grouped, weather_data_grouped, on='time_slot', how='inner')

print(orders_weather_data)


     time_slot  order_gap  temperature        pm25
0            0       5092     6.548387  133.838710
1            1       5678     6.176471  132.735294
2            2       6995     6.117647  131.911765
3            3       8847     6.088235  132.500000
4            4       9093     5.972222  132.305556
..         ...        ...          ...         ...
139        139       3463     6.387097  132.967742
140        140       3200     6.108108  129.540541
141        141       3620     5.970588  127.941176
142        142       4084     6.085714  125.971429
143        143       4039     6.125000  125.812500

[144 rows x 4 columns]


In [81]:
# merge the poi_list class characteristics with the cluster_map
# cluster_map: region_hash, region_id
# poi_data: district_hash, poi_ids
# merge on district_hash
cluster_map_poi = pd.merge(cluster_map, poi_data, left_on='region_hash', right_on='region_hash', how='inner')

# remove the region_hash column
cluster_map_poi = cluster_map_poi.drop(['region_hash'], axis=1)

print(cluster_map_poi.head())


   region_id                                            poi_ids
0          1  ['4#16:415', '1#10:498', '24:2324', '25:1245',...
1          2  ['4#16:166', '24:83', '25:581', '20:24153', '2...
2          3  ['20#7:830', '20#4:166', '20#2:166', '20#1:157...
3          4  ['4#16:249', '1#10:83', '24:581', '25:747', '2...
4          5  ['20#7:581', '20#5:166', '20#4:83', '8#4:332',...


In [88]:
# merge the orders_data with the cluster_map_poi

